# Monetary Policy Shocks & S&P 500 Reactions (2020–2025)

**Project Overview**  
This notebook analyses how Federal Reserve rate changes and FOMC announcements impact S&P 500 daily returns, then simulates hypothetical future shocks starting from the most recent data (Dec 30, 2025).

**Data Sources**  
- FRED: Effective Federal Funds Rate (DFF), M1 Money Supply  
- Yahoo Finance: S&P 500 (^GSPC)

**Methodology**  
1. Feature engineering (returns, rate changes, shock classification)  
2. OLS regression: SP500_Return ~ rate change + lags + shock dummies  
3. Simulation of future paths under hypothetical shocks (shock applied only on day 0)

In [1]:
import sys
from pathlib import Path
import pandas as pd

repo_root = Path().resolve().parent
sys.path.insert(0, str(repo_root))

In [2]:
from src.shocks_and_reactions.data_loader import DataLoader
from src.shocks_and_reactions.shock_events import IdentifyShockEvents
from src.shocks_and_reactions.simulate_events import EventSimulator

## 1. Data Loading & Visualising Past Shock Events

In [3]:
data_loader = DataLoader()
data_loader.load_data()

data_path = repo_root / "data" / "shocks_and_reactions" / "combined_data.csv"

data = pd.read_csv(data_path, parse_dates=["Date"], index_col="Date")

shock_identifier = IdentifyShockEvents(data)
shock_identifier.identify_shock_events()

                            OLS Regression Results                            
Dep. Variable:           SP500_Return   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     46.12
Date:                Sat, 10 Jan 2026   Prob (F-statistic):           1.60e-11
Time:                        14:23:25   Log-Likelihood:                 4402.0
No. Observations:                1505   AIC:                            -8800.
Df Residuals:                    1503   BIC:                            -8789.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0007      0.000      2.062

### Important Context: Why "Cut" Events Show Negative Cumulative Returns

The strongly negative average cumulative return for "Cut" events (blue line) may seem counter-intuitive — conventional wisdom suggests rate cuts should boost stocks.

However, this result is driven almost entirely by the two **emergency rate cuts in March 2020** (March 3 and March 15), when the Fed rapidly brought rates to near-zero in response to the COVID-19 market crash and economic shutdowns.

- In crisis environments, large rate cuts often signal severe distress, leading to continued selling pressure in the immediate event window.
- Markets front-ran bad news: sharp declines occurred in the days leading up to and around these announcements.
- Long-term, these cuts were highly supportive (S&P 500 more than doubled from the March 2020 low), but the short-term [-10, +20] day windows capture the panic phase.

The milder 25–50 bp cuts in 2024–2025 (September, November, December) occurred in a non-recessionary "soft landing" environment and were generally positive for risk assets — but there are too few of them relative to the 2020 outliers to offset the average in this sample.

**Key takeaway**: The stock market reaction to rate cuts depends heavily on **why** the Fed is cutting:
- **Crisis cuts** → short-term negative (bad news signal)
- **Growth-supportive / soft-landing cuts** → positive (lower discount rates, liquidity boost)

This nuance will be important when building forward-looking scenarios for 2026 policy.

## 2. Model Fitting

In [4]:
simulator = EventSimulator(
        data=Path(data_path)
    )
simulator.feature_engineering()
simulator.fit_model()

                            OLS Regression Results                            
Dep. Variable:           SP500_Return   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     11.48
Date:                Sat, 10 Jan 2026   Prob (F-statistic):           6.74e-11
Time:                        14:23:39   Log-Likelihood:                 4404.1
No. Observations:                1504   AIC:                            -8796.
Df Residuals:                    1498   BIC:                            -8764.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.0046    

Key insights:
- Strong mean reversion (negative lagged return coef)
- Hike dummy: large negative effect on announcement day
- Overall R² low (normal for daily returns)

## 3. Scenario Simulations (from Dec 30, 2025)

### Scenario 1: 25 bp Rate Hike (Hawkish Surprise)

In [5]:
hike = simulator.simulate_event(
        days_ahead=10, announcement_rate_change_bp=25.0, shock_type="Hike"
    )
# print(hike)
simulator.plot_simulation(hike, title="Simulation: 25bp Rate Hike")

Simulation starting from last available date: 2025-12-30 00:00:00
Starting S&P 500 level: 6,896.24
Starting lagged return: -0.003492
Starting lagged M1 growth: 0.000000


### Scenario 2: 25 bp Rate Cut (Dovish Surprise)

In [6]:
cut = simulator.simulate_event(
        days_ahead=10, announcement_rate_change_bp=-25.0, shock_type="Cut"
    )
# print(cut)
simulator.plot_simulation(cut, title="Simulation: 25bp Rate Cut")

Simulation starting from last available date: 2025-12-30 00:00:00
Starting S&P 500 level: 6,896.24
Starting lagged return: -0.003492
Starting lagged M1 growth: 0.000000


### Scenario 3: No Change (Base Case)

In [7]:
base = simulator.simulate_event(
        days_ahead=10, announcement_rate_change_bp=0.0, shock_type="No_Shock"
    )
# print(base)
simulator.plot_simulation(base, title="Simulation: No Shock")

Simulation starting from last available date: 2025-12-30 00:00:00
Starting S&P 500 level: 6,896.24
Starting lagged return: -0.003492
Starting lagged M1 growth: 0.000000


## 4. Key Takeaways & Limitations

- Hikes cause short-term pain but recover quickly in bull markets
- Cuts provide steady support
- No change leads to continued mild upside
- Limitations: Model assumes constant M1 growth, no volatility clustering, shock effect only on day 0
